### LSE Data Analytics Online Career Accelerator 

# DA301:  Advanced Analytics for Organisational Impact

## Demostration video: A/B testing using Python

#### *This is a possible solution to the demostration video.* 
In this video, we will perform A/B testing using Python on a real-life scenario. 

Consider the following scenario (adapted from Fillinich 2020):
An expanding e-commerce company wants to test a new version of its home page. The company has put together a new layout or design that they hope will increase its conversion rate. The conversion rate is a key metric, as it refers to the percentage of visitors that take a desired action on the website. In this case, that desired action is clicking through to the product page.
The company's previous home page had a conversion rate of only 13%. And the company is hoping that the new design will increase the conversion rate by at least 2%. For the business to meet their target here, the conversion rate needs to improve from 13% to at least 15%.

This is where a data analyst comes in: We can use an A/B test to determine if the conversion rate will indeed go up with 2%.

# 

## 1. Design and set up an A/B test

# 

## 2. Conduct a power analysis

In [1]:
# Import statsmodel for statistical calculations and 
# TTestIndPower class to calculate the parameters.
import statsmodels.stats.api as sms
from statsmodels.stats.power import TTestIndPower

# Specify the three required parameters for the power analysis:
alpha = 0.05 
power = 0.80 
effect = sms.proportion_effectsize(0.13, 0.15) 

# Perform power analysis by using the solve_power() function:
# Specify an instance of TTestIndPower.
analysis = TTestIndPower() 

# Calculate the sample size and list the parameters.
result = analysis.solve_power(effect, power=power, nobs1=None,
                              ratio=1.0, alpha=alpha) 

# Print the output.
print('Sample Size: %.3f' % result)

Sample Size: 4720.435


> What if we want to increase the power from 0.80 to 0.90?

In [4]:
# Import statsmodel for statistical calculations and 
# TTestIndPower class to calculate the parameters.
import statsmodels.stats.api as sms
from statsmodels.stats.power import TTestIndPower

# Specify the three required parameters for the power analysis:
alpha = 0.05 
power = 0.90 
effect = sms.proportion_effectsize(0.13, 0.15) 

# Perform power analysis by using the solve_power() function:
# Specify an instance of TTestIndPower.
analysis = TTestIndPower() 

# Calculate the sample size and list the parameters.
result = analysis.solve_power(effect, power=power, nobs1=None,
                              ratio=1.0, alpha=alpha) 

# Print the output.
print('Sample Size: %.3f' % result)

Sample Size: 6319.011


# 

## 3. Prepare the data in Python

In [5]:
# Install scipy.
!pip install scipy

In [6]:
# Import necessary libraries, packages and classes.
import pandas as pd
import math
import numpy as np
import statsmodels.stats.api as sms
import scipy.stats as st
import matplotlib as mpl
import matplotlib.pyplot as plt

In [8]:
# Read the CSV file (ab_data.csv).
df = pd.read_csv('../Data/ab_data.csv')

# View the DataFrame.
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [9]:
# Check the metadata.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [10]:
# Check for duplicates.
# Pandas's duplicated() function to check the user_id column. 
print(df[df.user_id.duplicated()])

        user_id                   timestamp      group landing_page  converted
2656     698120  2017-01-15 17:13:42.602796    control     old_page          0
2893     773192  2017-01-14 02:55:59.590927  treatment     new_page          0
7500     899953  2017-01-07 03:06:54.068237    control     new_page          0
8036     790934  2017-01-19 08:32:20.329057  treatment     new_page          0
10218    633793  2017-01-17 00:16:00.746561  treatment     old_page          0
...         ...                         ...        ...          ...        ...
294308   905197  2017-01-03 06:56:47.488231  treatment     new_page          0
294309   787083  2017-01-17 00:15:20.950723    control     old_page          0
294328   641570  2017-01-09 21:59:27.695711    control     old_page          0
294331   689637  2017-01-13 11:34:28.339532    control     new_page          0
294355   744456  2017-01-13 09:32:07.106794  treatment     new_page          0

[3894 rows x 5 columns]


In [11]:
# Drop duplicate values.
# Use drop_duplicates to return the Series without the duplicate values.
df2 = df.drop_duplicates(subset = 'user_id') 

# Check the metadata.
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290584 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       290584 non-null  int64 
 1   timestamp     290584 non-null  object
 2   group         290584 non-null  object
 3   landing_page  290584 non-null  object
 4   converted     290584 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 13.3+ MB


In [12]:
# Remove unnecessary columns.
# Use dropped.drop to remove irrelevant columns from the DataFrame. 
# Specify that user_id and timestamp are columns (i.e. axis 1). 
df3 = df2.drop(['user_id', 'timestamp'], axis=1)  

# Check the DataFrame.
df3.head()

,group,landing_page,converted
0,control,old_page,0
1,control,old_page,0
2,treatment,new_page,0
3,treatment,new_page,0
4,control,old_page,1


In [13]:
# Check for errors.
# Use crosstab to compute a simple cross-tabulation between two variables.
pd.crosstab(df3['group'], df3['landing_page'])

landing_page,new_page,old_page
group,,
control,1006,144226
treatment,144314,1038


In [14]:
# Specify groups to be dropped.
df4 = df3[((df3.group == 'control') & (df3.landing_page == 'old_page')) | (
    (df3.group == 'treatment') & (df3.landing_page == 'new_page'))]

# Print the shape of the new final table.
print(df4.shape)
df4['group'].value_counts()

(288540, 3)


treatment    144314
control      144226
Name: group, dtype: int64

In [15]:
# Re-check/compute another simple cross-tabulation.
pd.crosstab(df4['group'], df4['landing_page'])

landing_page,new_page,old_page
group,,
control,0,144226
treatment,144314,0


# 

## 4. Perform random sampling with Pandas

In [18]:
# Obtain a simple random sample for control and treatment groups with n = 4721; 
# set random_stategenerator seed at an arbitrary value of 22.
# Obtain a simple random sample for the control group.
control_sample = df4[df4['group'] == 'control'].sample(n=4721, 
                                                       random_state=22) 

# Obtain a simple random sample for the treatment group.
treatment_sample = df4[df4['group'] == 'treatment'].sample(n=4721,
                                                           random_state=22)

control_sample

,group,landing_page,converted
58605,control,old_page,0
197229,control,old_page,0
228637,control,old_page,0
147532,control,old_page,0
19562,control,old_page,0
...,...,...,...
132071,control,old_page,0
282023,control,old_page,0
175736,control,old_page,0
273069,control,old_page,0


In [27]:
# Join the two samples.  
ab_test = pd.concat([control_sample, treatment_sample], axis=0)  

# Reset the A/B index.
ab_test.reset_index(drop=True, inplace=True) 

# Print the sample table.
ab_test  

,group,landing_page,converted
0,control,old_page,0
1,control,old_page,0
2,control,old_page,0
3,control,old_page,0
4,control,old_page,0
...,...,...,...
9437,treatment,new_page,0
9438,treatment,new_page,0
9439,treatment,new_page,0
9440,treatment,new_page,0


# 

## 5. Analyse the data

In [33]:
# Calculate basic statistics.
# Import library.
# SEM stands for standard error mean.
from scipy.stats import sem

# Group the ab_test data set by group and aggregate by converted.
conversion_rates = ab_test.groupby('group')['converted']


# Calculate conversion rates by calculating the means of columns STD_p and SE_p.
conversion_rates = conversion_rates.agg([np.mean, np.std,sem])

# Assign names to the three columns.
conversion_rates.columns = ['conversion_rate',
                            'std_deviation',
                            'std_error']  

# Round the output to 3 decimal places.
conversion_rates.style.format('{:.3f}')

,conversion_rate,std_deviation,std_error
group,,,
control,0.124,0.330,0.005
treatment,0.115,0.319,0.005


In [43]:
# Calculate statistical significance.
# Import proportions_ztest and proportion_confint from statsmodels.
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

# Create a subset of control and treatment results.
control_results = ab_test[ab_test['group'] == 'control']['converted']
treatment_results = ab_test[ab_test['group'] == 'treatment']['converted']

# Determine the count of the control_results and 
# treatment_result sub-data sets and store them in their respective variables.
n_con = control_results.count()
n_treat = treatment_results.count()

# Create a variable 'success' with the sum of the two data sets in a list format. 
successes = [control_results.sum(), treatment_results.sum()]

# Create a variable 'nobs' which stores the values of 
# variables n_con and n_treat in list format. 
nobs = [n_con, n_treat] 

# Use the imported libraries to calculate the statistical values. 
z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes,
                                                                        nobs=nobs,
                                                                        alpha=0.05)

# Print the outputs (with lead-in text). The .3f indicates the number of decimal places.
print(f"Z test stat: {z_stat:.3f}")
print(f"P-value: {pval:.3f}")
print(f"Confidence Interval of 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]")
print(f"Confidence Interval of 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]")

Z test stat: 1.396
P-value: 0.163
Confidence Interval of 95% for control group: [0.115, 0.134]
Confidence Interval of 95% for treatment group: [0.106, 0.124]


# 

## 6. Conclusion

Our results indicated a $p=0.163$ (16.3%) with an $alpha=0.05$. Therefore, we cannot reject the $H_0$. Why? Recall that $p$-values summarise statistical differences, essentially calculating the probability that the data could deviate from $H_0$. A $p=0.163$ (16.3%) suggests that the new design did not perform better than the old design. The confidence interval for the treatment group does not reach 13%, never mind the desired 15%, confirming that the new website design was altogether ineffective in increasing the conversion rate.

Although the website redesign did not result in a higher conversion rate, the result of the test does not necessarily need to be seen as a failure. The non-rejection of the $H_0$ simply means that there isn’t enough statistical evidence to disprove the hypothesis. In theory, we can repeat the test with a different sample size, or a change in the confidence level, and so on. 

And, despite the result, additional data has been gained! The e-commerce company now knows that maybe it isn’t the homepage design that is discouraging users from taking desired actions. There could also be invalid assumptions in the test itself. With this in mind, the company can regroup and reconsider how to either improve the conversion rate or run more tests.